## Data exploration notebook
This notebook uses cleanvision from cleanlab to find issues in the CUB-200-2011 bird images dataset. In addition, problematic samples are written to a .csv file such that these can be filtered out during preprocessing.

In [ ]:
!pip install git+https://github.com/cleanlab/cleanvision.git

In [ ]:
# Imports
from cleanvision.imagelab import Imagelab
import os
import numpy as np
from numpy import genfromtxt

In [ ]:
# Find issues in the whole dataset
# Directory where all images are stored
rootdir = './data/CUB_200_2011/images/merge_folder'
# Find issues with imagelab
imagelab = Imagelab(data_path=rootdir)
imagelab.find_issues()


In [5]:
# Summarize issues found
imagelab.issue_summary

,issue_type,num_images
0,blurry,15
1,near_duplicates,10
2,grayscale,8
3,low_information,3
4,exact_duplicates,2
5,light,1
6,odd_aspect_ratio,1
7,dark,0


In [ ]:
# Visualize some near duplicates
near_dup_images = imagelab.issues[imagelab.issues["is_near_duplicates_issue"] == True]
near_dup_image_files = near_dup_images.index.tolist()
print(near_dup_image_files)
imagelab.visualize(image_files=near_dup_image_files)

In [ ]:
# Get lists of exact and near duplicate sets
exact_dups_list = imagelab.info['exact_duplicates']['sets']
near_dups_list = imagelab.info['near_duplicates']['sets']
# Concatenate the lists
samples_list = exact_dups_list + near_dups_list
print(samples_list)
print(samples_list[0][0])
print(exact_dups_list[0][0])

In [43]:
def problematic_samples_tocsv(samples_list : list) -> None:
    """
    Gets the first problematic samples from the sample list and writes the file names to a csv file.

    Parameters
    -----------
        sample_list : list
            List containing sets of problematic samples.
    """
    problematic_samples = []
    for i in range(len(samples_list)):
        path = samples_list[i][0]
        head_tail = os.path.split(path)
        tail = head_tail[1]
        problematic_samples.append(tail)

    np.savetxt("problematic_samples.csv", problematic_samples, delimiter=",", fmt ='% s')
    print(problematic_samples)
    print(len(problematic_samples))


In [44]:
problematic_samples_tocsv(samples_list=samples_list)

['Pigeon_Guillemot_0018_40195.jpg', 'Bronzed_Cowbird_0025_796213.jpg', 'Chuck_Will_Widow_0042_796983.jpg', 'Red_Legged_Kittiwake_0020_795439.jpg', 'Summer_Tanager_0047_140164.jpg', 'Swainson_Warbler_0022_794868.jpg']
6


In [45]:
# Load problematic samples from csv
laoded_problematic_samples = np.loadtxt('problematic_samples.csv', delimiter=',', dtype=str )
print(laoded_problematic_samples)

['Pigeon_Guillemot_0018_40195.jpg' 'Bronzed_Cowbird_0025_796213.jpg'
 'Chuck_Will_Widow_0042_796983.jpg' 'Red_Legged_Kittiwake_0020_795439.jpg'
 'Summer_Tanager_0047_140164.jpg' 'Swainson_Warbler_0022_794868.jpg']


In [60]:

# Test code for crop_imgages.py
# To filter out the problematic samples

boxes = open("./data/CUB_200_2011/bounding_boxes.txt")
images_id = open("./data/CUB_200_2011/images.txt")
train_test = open("C./data/CUB_200_2011/train_test_split.txt")



for (line, box, train) in zip(images_id, boxes, train_test):
    i = 0
    image_path = line.split()[1]
    image_name = image_path.split("/")[1]
    is_train = int(train.split()[1])

    if is_train and not(image_name in laoded_problematic_samples):
        i+=1
    elif not (is_train) and not(image_name in laoded_problematic_samples):
        i+=2
    else: 
        print("problematic samples skipped in train test split: ", image_name)


problematic samples skipped in train test split:  Chuck_Will_Widow_0042_796983.jpg
problematic samples skipped in train test split:  Bronzed_Cowbird_0025_796213.jpg
problematic samples skipped in train test split:  Pigeon_Guillemot_0018_40195.jpg
problematic samples skipped in train test split:  Red_Legged_Kittiwake_0020_795439.jpg
problematic samples skipped in train test split:  Summer_Tanager_0047_140164.jpg
problematic samples skipped in train test split:  Swainson_Warbler_0022_794868.jpg
